# Vertex AI RAG Engine にベクトル DB 作成


In [ ]:
from vertexai import rag

# from vertexai.generative_models import GenerativeModel, Tool
import vertexai

import os
from dotenv import load_dotenv

load_dotenv()

True

# コーパスの作成

- RAG Engine 作成時、デフォルトのベクトル DB は、RagManagedDb が選択される
- ほかにもベクトル DB として pinecone などを選択できる


In [5]:
# Get Vertex AI configuration from environment
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

# Initialize Vertex AI at package load time
try:
    if PROJECT_ID and LOCATION:
        print(f"Initializing Vertex AI with project={PROJECT_ID}, location={LOCATION}")
        vertexai.init(project=PROJECT_ID, location=LOCATION)
        print("Vertex AI initialization successful")
    else:
        print(
            f"Missing Vertex AI configuration. PROJECT_ID={PROJECT_ID}, LOCATION={LOCATION}. "
            f"Tools requiring Vertex AI may not work properly."
        )
except Exception as e:
    print(f"Failed to initialize Vertex AI: {str(e)}")
    print("Please check your Google Cloud credentials and project settings.")

Initializing Vertex AI with project=adk-rag-first, location=us-central1
Vertex AI initialization successful


In [ ]:
# Embedding modelの設定
# 日本語なので多言語モデルを選択
embedding_model_config = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-multilingual-embedding-002"
    )
)

In [ ]:
# コーパスの作成
display_name = "test_corpus"
rag_corpus = rag.create_corpus(
    display_name=display_name,
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=embedding_model_config,
    ),
)

print(f"rag corpus name: {rag_corpus.name}")

rag corpus name: projects/522424496787/locations/us-central1/ragCorpora/137359788634800128


In [9]:
# RAG対象ファイルのインポート
paths = [
    "gs://test_bucket_for_access_check/merge_tags_content.txt",
]
rag.import_files(
    corpus_name=rag_corpus.name,
    paths=paths,
    transformation_config=rag.TransformationConfig(
        chunking_config=rag.ChunkingConfig(
            chunk_size=256,
            chunk_overlap=50,
        ),
    ),
    max_embedding_requests_per_min=1000,
)

imported_rag_files_count: 1

# RAG 検索試し

- score は、コサイン類似度を利用し、『1-cos(Θ)』値なので値が小さいほど類似している
- score の範囲：0~2（0 に近いほど類似）


In [10]:
# RAG corpusのパスをセット
rag_corpus_path = rag_corpus.name

In [ ]:
# query実行

query = "tags:貸付・融資, content:農林漁業にたいするコロナウイルス対策支援について"

response = rag.retrieval_query(
    rag_resources=[rag.RagResource(rag_corpus=rag_corpus_path)],
    text=query,
    rag_retrieval_config=rag.RagRetrievalConfig(
        top_k=3,
    ),
)

In [12]:
response

contexts {
  contexts {
    source_uri: "gs://test_bucket_for_access_check/merge_tags_content.txt"
    text: "1人の雇用につき20万円の助成金を支給します。\r\ntags:貸付・融資, content:新型コロナウイルス感染症対応農林漁業特別対策資金新型コロナウイルス感染症により、事業活動に影響を受けている農林漁業者等の方々に向けた運転資金の融資で、農林漁業の各近代化資金の枠組みにより実施するものです。"
    source_display_name: "merge_tags_content.txt"
    score: 0.1421343965535613
    chunk {
      text: "1人の雇用につき20万円の助成金を支給します。\r\ntags:貸付・融資, content:新型コロナウイルス感染症対応農林漁業特別対策資金新型コロナウイルス感染症により、事業活動に影響を受けている農林漁業者等の方々に向けた運転資金の融資で、農林漁業の各近代化資金の枠組みにより実施するものです。"
    }
  }
  contexts {
    source_uri: "gs://test_bucket_for_access_check/merge_tags_content.txt"
    text: "tags:貸付・融資, content:新型コロナウイルス感染症対応融資（伴走全国）新型コロナウイルス感染症の影響を受け、売上等が減少している都内中小企業の方々が、金融機関との対話を通じて策定する経営行動計画書に基づき、金融機関による継続的な伴走支援を受けながら、経営改善等を図る場合にご利用いただけます。＜融資限度額＞：6,"
    source_display_name: "merge_tags_content.txt"
    score: 0.18051889404561183
    chunk {
      text: "tags:貸付・融資, content:新型コロナウイルス感染症対応融資（伴走全国）新型コロナウイルス感染症の影響を受け、売上等が減少している都内中小企業の方々が、金融機関との対話を通じて策定する経営行動計

In [ ]:
result = [
    {
        "content": ctx.text,  # 取り出したチャンク本文
        # "uri": ctx.source_uri,  # 元ファイルの GCS URI など
        "score": ctx.score,  # コサイン距離 (小さいほど類似)
    }
    for ctx in response.contexts.contexts
]

result

[{'content': '1人の雇用につき20万円の助成金を支給します。\r\ntags:貸付・融資, content:新型コロナウイルス感染症対応農林漁業特別対策資金新型コロナウイルス感染症により、事業活動に影響を受けている農林漁業者等の方々に向けた運転資金の融資で、農林漁業の各近代化資金の枠組みにより実施するものです。',
  'uri': 'gs://test_bucket_for_access_check/merge_tags_content.txt',
  'score': 0.1421343965535613},
 {'content': 'tags:貸付・融資, content:新型コロナウイルス感染症対応融資（伴走全国）新型コロナウイルス感染症の影響を受け、売上等が減少している都内中小企業の方々が、金融機関との対話を通じて策定する経営行動計画書に基づき、金融機関による継続的な伴走支援を受けながら、経営改善等を図る場合にご利用いただけます。＜融資限度額＞：6,',
  'uri': 'gs://test_bucket_for_access_check/merge_tags_content.txt',
  'score': 0.18051889404561183},
 {'content': '000円（税込）で利用可能\r\ntags:貸付・融資, content:新型コロナウイルス感染症の影響を受けた休業や失業等による緊急小口資金、総合支援資金（特例貸付）新型コロナウイルス感染症の影響を受け、収入の減少や失業・休業などにより生活資金でお困りの方に、無利子で貸付を行います。1.',
  'uri': 'gs://test_bucket_for_access_check/merge_tags_content.txt',
  'score': 0.1849476174047644}]